<a href="https://colab.research.google.com/github/aasokolova/socnet_course/blob/main/Simple_contagion_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AS, 25.02.2025**


*SocNet in SocSciRes: Session 3*

1. Import necessary packages

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import imageio

2. Create a graph: core-periphery network (Option 2)

In [ ]:
G = nx.Graph()

### Define core nodes
core_nodes = [f'C{i}' for i in range(6)]
for node in core_nodes:
    G.add_node(node, type='core')

### Define peripheral nodes with intra-cluster ties
peripheral_nodes = {}
for i, core in enumerate(core_nodes):
    peripheral_nodes[core] = [f'P{i}_{j}' for j in range(5)]
    for p in peripheral_nodes[core]:
        G.add_node(p, type='periphery')
        G.add_edge(core, p)  # Connect periphery to core
    # Add intra-cluster ties between peripheral nodes
    for p1 in peripheral_nodes[core]:
        for p2 in peripheral_nodes[core]:
            if p1 != p2:
                G.add_edge(p1, p2)

### Connect core nodes
core_edges = [
    ('C0', 'C1'), ('C1', 'C2'), ('C2', 'C3'), ('C3', 'C4'), ('C4', 'C5'), ('C5', 'C0'),
    ('C0', 'C2'), ('C2', 'C4'), ('C1', 'C3'), ('C3', 'C5')
]
G.add_edges_from(core_edges)

### Define colors: 'Brick red' for core, 'Navy'  for periphery
color_map = {'core': '#B22222', 'periphery': '#001F3F'}

### Plot the network
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G, seed=42)
node_colors = [color_map[G.nodes[n]['type']] for n in G.nodes]
nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=300, edge_color='gray', font_size=8)
plt.title("Core-Periphery Network")
plt.show()

3. Run a simple contagion simulation

In [ ]:
### Reset node states
for node in G.nodes:
    G.nodes[node]['state'] = 0
initial_adopter = 'C0'
G.nodes[initial_adopter]['state'] = 1

### Define my own contagion model parameters
adoption_prob = 0.3  # Probability of adoption per step
steps = 10  # Maximum steps to run simulation

### Simple diffusion step
def simple_diffusion_step(G, adoption_prob):
    new_adopters = []
    for node in G.nodes:
        if G.nodes[node]['state'] == 0:  # if not yet adopted (state 0)
            neighbors = list(G.neighbors(node))
            adopted_neighbors = sum(G.nodes[n]['state'] for n in neighbors)
            if adopted_neighbors > 0 and random.random() < adoption_prob:
                new_adopters.append(node)
    for node in new_adopters:
        G.nodes[node]['state'] = 1
    return len(new_adopters) > 0  # Loop stops if no new adoptions occur

4. Create a GIF

In [ ]:
frames = []
pos = nx.spring_layout(G, seed=42)  # Fixed layout for consistency
adopted_color = "#CDA434"  # Mustard yellow color for adoptee nodes
duration_per_frame = 2.0  # Slower animation

for step in range(steps):
    if not simple_diffusion_step(G, adoption_prob):
        break
    fig, ax = plt.subplots(figsize=(8, 6))
    node_colors = [color_map[G.nodes[n]['type']] if G.nodes[n]['state'] == 0 else adopted_color for n in G.nodes]
    nx.draw(G, pos, node_color=node_colors, node_size=300, edge_color='gray')
    ax.set_title(f"Simple Contagion - Step {step+1}")
    frame_path = f"simple_diffusion_step_{step+1}.png"
    plt.savefig(frame_path)
    plt.close(fig)
    frames.append(frame_path)

### Create GIF from frames
gif_path = "simple_diffusion.gif"
imageio.mimsave(gif_path, [imageio.imread(frame) for frame in frames], duration=duration_per_frame)

### File path for download
print(f"GIF saved as: {gif_path}")